In [ ]:
import os 

import pandas as pd
import numpy as np
import scipy.stats
from sklearn.linear_model import LinearRegression
import sklearn.preprocessing
import sklearn.metrics

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
import seaborn as sns

import top2000analysis

In [ ]:
FOLDER_OUTPUT = os.path.join('..', 'Output')
FOLDER_DATA = os.path.join('..', 'Data')

Recently, I published a [blog](https://sjoerdcornelissen.com/2021/03/02/the-effect-of-passing-away-on-top-2000-ranking/.) on the effect of passing away on rakings in the Top 2000. There, I dived deep into the outcomes of the model. Additionally, I also want to show how to build such a model and do the analysis, which I will do in this blog series.

I will do so in three parts:
1. *Part 1: Reading and understanding the data*  
  In this part, I will acquire the data and get a first feel for it.
1. *Part 2: Building and checking a hierarchical Bayesian regresson model*  
In this part, we will use the data and knowledge about it we acquired in the first part to actually build a hierachical, Bayesian model. Part of building a Bayesian model is also checking whether it works as expected.
1. *Part 3: Using the model for inference and prediction*  
  In the final part, I will show how to use the model to get the insights we want.

THIS NOTEBOOK IS STILL UNDER CONSTRUCTION AND WILL BE UPDATED

# Reading and understanding the Top 2000 data 

No data science without data. While it is often the least sexy part, acquiring, structuring and importantly understanding the data well will set a project up for success, while skimping this might look good in the short term, but always turns out to hit you hard much quicker than expected, when weird results keep popping up and find out whether this is a problem with the data, the code or there is actually something interesting going up turns into an almost impossible task.

In this part, we will do three things:
1. _Acquiring the data_  
We scrape wikipedia for everything about the Top 2000
1. _Structuring the data_  
By putting the data into tables that are structured through a data model, we keep all the flexibility for future analyses.
1. _Understanding the data_  
By looking into the univariate correlations, we get a feeling for what influences what, which will help us when building a model.

## Acquiring the data: scraping wikipedia
First, we scrape all the data from Wikipedia. While NPO Radio 2, the organizer, also publishes the list in Excel form on the [Top 2000 website](https://www.nporadio2.nl/top2000), I did not use that for two reasons:
* It uses an inconsistent format through the years, both in column names and band names.
* For the more detailed artist information I need, I would have to use an external source anyway, so it is easier to immediately use Wikipedia.

Wikipedia has complete rankings of all the previous Top 2000, directly with links to more detailed pages about the artists and songs: https://nl.wikipedia.org/wiki/Lijst_van_Radio_2-Top_2000%27s.

While there are libraries that access Wikipedia in python, such as [`wikipedia`](https://pypi.org/project/wikipedia/) and [`Wikipedia-API`](https://pypi.org/project/Wikipedia-API/), I did not use those:
* They are generally oriented towards a user-oriented way of looking through Wikipidia, with a user searching and reading the main text. I, on the other hand, am focused on extracting the Tables and Infoboxes, and there was no good support for that. 
* The link(s) that are in the cells are important, since we will want to browse those for more detailed information. This generally does not come out of the box.

Therefore, I decided to write a very quick module that reads Tables and Infoboxes from Wikipedia. It can be found on my Github as [`wikipediareader`](https://github.com/SjoerdCor/Top2000/blob/main/src/wikipediareader.py).

This module is used by the `readtop2000` module, which reads and then writes the information we need about the Top2000
1. Scrapes the full list (https://nl.wikipedia.org/wiki/Lijst_van_Radio_2-Top_2000%27s)
1. Goes to all artist pages it finds on the full list and adds the information from the Infobox on that page to the table
1. Cleans the column names and types and validates the reading worked
1. Writes the information to a structured data model.

We store all the tables as _.parquet_-files, a commonly used format that also store data types and is memory efficient and quick to read and write.

## Structuring the data
### Splitting data into a data model
By thinking about how we should structure and save the data, we save ourselves from a lot of pain down the road. I decided to save the information according to the following data model:

<img src="../blogfigures/Reading and understanding Top 2000 data/datamodel.PNG" title="The data model consists of ranking table with a many-to-one relationship to song, which has a one to many relationship to songartist, which has a many-to-one relationship with artist"/>
We have the following entities:

* A **ranking** for each song for each year in the Top 2000
* A **song** which is performed by one or more artists
* An **artist** who performs one or more songs.

The most notable as aspect is that **song** and **artist** have a [many-to-many relationship](https://en.wikipedia.org/wiki/Many-to-many_(data_model)), so we add a junction table between those. The artist table contains a column for every label that is used on at least one artist, so it has a lot columns and a lot of `NaN`s.

Now that we have read and structured all the data, we can finally get the data:

In [ ]:
%%time
import readtop2000
import importlib; importlib.reload(readtop2000)
downloader = readtop2000.Top2000Downloader()
downloader.download_and_write(FOLDER_OUTPUT)

### Recombining and selecting the data
Now, we wish to get a table on which we can perform the analysis. For this, we combine all data, with one row for each artist for each song, so a song which is a duet occurs twice. 

In [ ]:
import importlib; importlib.reload(top2000analysis)

In [ ]:
a = top2000analysis.AnalysisSetCreator(FOLDER_DATA)

df_full = a._combine_data()
print(f"In its history, the Top 2000 has seen {df_full['SongID'].nunique()} unique songs and {df_full['ArtistID'].nunique()} unique artists")

For our analysis, we only select the rows of the years in which the artist passed away, and add the information of the year before, so we can compare the number of votes.

We also calculate the percentage of the vote each song received every year based on a model from [Peter Meindertsma](https://www.petermeindertsma.nl/blog/benadering-aantal-stemmen-per-liedje-in-de-top-2000-van-2014/). We end up with the dataframe as shown below, which contains all the information we need to perform the analysis.

In [ ]:
df_artist = a.create_artist_set()

In [ ]:
df = a.create_full_feature_set()

In [ ]:
with pd.option_context('display.max_columns', None):
    display(df.sample(5))

In [ ]:
print(f'The final data contains {len(df)} rows, which means there are {len(df)} songs which were in the Top 2000 the year before the artist passed away.')
print(f'They come from {df["ArtistID"].nunique()} unique artists')

## Basic data inspection

In [ ]:
ax = df['Year'].value_counts().sort_index().plot(ylim=(0, None), xlabel='Year', ylabel='Nr of artists\nwho passed away')
ax.xaxis.set_major_locator(mtick.MaxNLocator(integer=True))
plt.show()

In [ ]:
ax = df['Year'].value_counts().sort_index().cumsum().plot(ylim=(0, None), xlabel='Year', ylabel='Nr of artists\nwho passed away')
ax.xaxis.set_major_locator(mtick.MaxNLocator(integer=True))
plt.show()

We see a number of artists passing awayt each year, with a spike in 2016.

Next, let's see what happened to the ranking of each song.

In [ ]:
ax = df.plot(kind='scatter', x='RankLastYear', y='Rank', label='Song of artist who passed away')
ax.plot([0, 2000], [0, 2000], 'k--', label='Position last year')
ax.set_xlabel('Ranking before passing away')
ax.set_ylabel('Ranking in year of passing away')
plt.legend()
plt.show()

Since most dots are below the dashed line, indeed almost all songs (but notably not all) receive some sort of boost in the year the artist passed away. We also see there are huge differences in the size of the effect: with some songs ranking below 1500 entering the Top 250 a year later, while some Top 500 songs stay right where they are.

For the analysis, the discrete nature of rankings is a problem; the difference between songs ranked 2000 and 1980 is very different from the songs ranked 21st and first. Therefore, we estimate the number of votes through a model developed by [Peter Meindertsma](https://www.petermeindertsma.nl/blog/benadering-aantal-stemmen-per-liedje-in-de-top-2000-van-2014/).
Furthermore, we define a _Boost_  as the ratio between the percentage of votes of the song in the year of the artist passing away and the percentage of the vote of that song in the year before.

In [ ]:
def plot_distribution(series, dist='norm', **kwargs):
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    series.plot(kind='kde', ax=axes[0], **kwargs)
    scipy.stats.probplot(series, dist=dist, plot=axes[1])
    plt.show()
    display(series.describe())

In [ ]:
plot_distribution(df['BoostSong'], xlim=(0, None))

We see the average song gets a boost of 2.3x as many votes as the year before, and the median is 1.75x as many votes. Notably, the Boost distrution is not Normal. Therefore, we also inspect the logarithm of the boost:

In [ ]:
plot_distribution(df['LogBoost'])

Still not perfectly normal, but much better. In our further estimation, we will often work with the logarithm, because the regular boost is dominated by a few outliers.

Next, we will also investigate the _artist boost_, defined as the the total number of votes for an artist in the year of his death divided by the total number of votes in the year before, instead of each song individually

In [ ]:
plot_distribution(df_artist['Boost'], xlim=(0, None))

Again, we see the same distribution. Notably, the average _artist_ boost is lower than the average _song_ boost. This can be explained if artists with multiple songs on average see a larger boost. We make the same plot, but now binned for easier interpretation, to show in the blog. 

In [ ]:
def plot_waterfall(data, color=None, buildup=False, **kwargs):
    '''
    Plot a buildup or builddown waterfall chart from data
    This function was adapted from https://pbpython.com/waterfall-chart.html

    Parameters
    ----------
    data: pd.Series to be shown as waterfall
    color: optionally give color as a list for each bar (to highlight some bars)
    buildup: False (default) for builddown, True for buildup

    Returns
    -------
    ax: Axis object
    data: the data, including a "total"-row
    blank: the size of the blank space before each bar
    '''
    if color is None:
        color = ['lightgray'] * len(data)

    blank = data.cumsum().shift(1).fillna(0)
    total = data.sum()
    data.loc['Total'] = total
    blank.loc['Total'] = 0
    color = color + ['gray']
    
    step = blank.reset_index(drop=True).repeat(3).shift(-1)
    step[1::3] = np.nan
    
    if buildup:
        data = data[::-1]
        blank = blank[::-1]
        color = color[::-1]

    ax = data.plot(kind='barh', stacked=True, left=blank, color=color, **kwargs)
    return ax, data, blank

data = pd.cut(df_artist['Boost'], [0, 1, 1.25, 1.5, 2.5, np.inf],
               labels=['No boost',
                       'Up to 25% more votes',
                       'Up to 50% more votes',
                       '1.5 - 2.5 x\nas many votes',
                       'More than 2.5\nx as many votes']).value_counts(normalize=True).sort_index(ascending=False)
data.index = data.index.astype(str)
ax, _, _ = plot_waterfall(data, color=['purple', 'purple', 'purple', 'purple', 'lightgray', 'gray'])
ax.set_ylabel('Effect size')
ax.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)

# plt.gcf().savefig(os.path.join(FOLDER_OUTPUT,'BoostDistribution.png'))

Now, let's check who got the smallest and largest boosts

In [ ]:
display(df_artist.query('Boost < 1')[['Name', 'Boost', 'LogPopularityNorm']].sort_values('Boost'))
display(df_artist.nlargest(10, 'Boost')[['Name', 'Boost', 'LogPopularityNorm']])

# Univariate analysis

To get a first feeling for the data, formulate hypotheses on what affects the size of the boost on three levels:
1. A __base__ effect for every artist
1. An __artist specific__ effect, based on the characteristics of the artist and his/her death - aspects that have to do with how prominent the artist will be in the news after his/her death
1. A __song specific__ effect, for which songs channel the newfound votes.

For each step, the hypotheses are shown in the table below:

<img src="../blogfigures/Hypothesis.png" alt="Hypotheses are formulated for each level, to predict the boost of a song" width="750"/>

In this section, we want to get a first look into the univariate 

Before we dive in, lets check the relative importance of the levels. For this, we look into artist who have multiple songs, and compare the explained variance score for the song boost as explained by the _artist_ boost on the one hand, and the _Relative Song_ boost (the boost of the song divided by the artist boost) on the other hand:

In [ ]:
df_multiple_songs = df.query('NrsBeforeDeath >= 2')
print(f'There are {len(df_multiple_songs)} songs by artists with at least two songs')
artist_variance = sklearn.metrics.explained_variance_score(df_multiple_songs['LogBoost'], np.log(df_multiple_songs['BoostArtist']))
song_variance = sklearn.metrics.explained_variance_score(df_multiple_songs['LogBoost'], df_multiple_songs['LogRelativeBoost'])
print(f'The explained variance of the artist is {artist_variance: .1%}')
print(f'The explained variance of the relative song boost is {song_variance: .1%}')

In [ ]:
ax = sns.violinplot(x='NrsBeforeDeath', y='SongRelativeBoost', data=df)
ax.set_ylim(0,2)

The artist is most important, but we also see there is definetly still a lot of variance within the songs of an artist, which we should try to model.

In [ ]:
def plot_with_trend(df, column, logy=False):
    ycolname = 'LogBoost' if logy else 'Boost'
    preds = LinearRegression().fit(df[[column]], df[ycolname]).predict(df[[column]])
    ax = df.plot(x=column, y=ycolname, kind='scatter', label='Passed away artists', c='grey')
    ax.plot(df[column], preds, 'k', label='Trend')
    return ax

## Artist level

### Artist age
Newsworthiness of the death of an artist depends on the circumstances. Specifically, artists dying young is something that attracts a lot of media attention. This happens a lot in our sample:

In [ ]:
pct_dying_before_80 = df_artist['AgePassing'].lt(80).mean()
print(f'{pct_dying_before_80:.1%} of the artists who passed away were younger than 80')

In [ ]:
ax = plot_with_trend(df_artist, 'AgePassing', logy=True)

We do not see a strong _univariate_ effect, but this may be because of collinearity with other effects

### Jaar
We hypothesize the boost has grown stronger over the years, because of more prevalent news.

In [ ]:
ax = plot_with_trend(df_artist, 'JaarTop2000', logy=True)
ax.xaxis.set_major_locator(mtick.MaxNLocator(integer=True))
plt.legend()
plt.tight_layout()
ax.set_frame_on(False)

### Nationality
We hypothesize Dutch voters care more about Dutch artists, out of chauvinism. The effect is hard to detect, since only a small number of Dutch artists passed away

In [ ]:
print(f'{df_artist["IsDutch"].sum()} Dutch artists passed away')

In [ ]:
_ = sns.catplot(x='IsDutch', y='LogBoost', data=df_artist)

### Popularity
When more popular artists die, they get much more news coverage: a relatively unknown artist may get a small article on page 15, while superstars may get entire TV shows devoted to hem. This in turn would result in much more prominent boosts for more popular artists

In [ ]:
ax = plot_with_trend(df_artist, 'PctVotes', logy=True)

This seems to be the case, but the effect between the _logarithm_  of the boost and the percentage of the votes is clearly non-linear, and the trendline is dominated by a few superstars. Let's look into the logarith of the percentage of votes

In [ ]:
plot_with_trend(df_artist, 'LogPopularity', logy=True)

This looks much better: there is a strong, mostly linear relation between the logarith of the boost and the logarithm of the popularity.

### Recency
We hypothesize that the more top of mind a death is, the more of a boost an artist can expect - so more recent deaths during the time of voting should get a larger boost

In [ ]:
ax = plot_with_trend(df_artist.query('DaysToStemperiode > -365'), 'DaysToStemperiode', logy=True)

Indeed, this is the case; however, it is not entirely clear if this is a linear relationship. Lets bucket the observations to see that clearer

In [ ]:
recency_buckets = (pd.cut(df_artist['DaysToStemperiode'].clip(lower=-365), 6, labels=False, retbins=False, right=False)
                  .map({i: v for i, v in enumerate(range(-333, 0, 60))}))

In [ ]:
df_artist.groupby(recency_buckets)['LogBoost'].agg(['mean', 'sem', 'std', 'count'])

In [ ]:
ax = df_artist.groupby(recency_buckets)['LogBoost'].agg(['mean', 'sem', 'std', 'count']).plot(y='mean', yerr='sem')
plt.xlim(-365, 0)
plt.show()

There appears to be little difference out to ~ 150 days, and then the boost grows much stronger.

### Recency of last hit

Finally, there might be an effect whether artists were recently popular

In [ ]:
ax = plot_with_trend(df_artist, 'YearsSinceLastHit', logy=True)

### Correlations

We find that, probably mostly by coincidence, in recent years artists in the Top 2000 have tended to pass away at later ages, which may partly explain why we do not see a strong age effect in a univariate distribution. Furthermore, we see that in recent years more popular artists have tended to pass away, which may spuri

In [ ]:
def plot_correlations(df, figsize=(12, 12)):
    fig, ax = plt.subplots(figsize=figsize)
    ax = sns.heatmap(df.corr(), cmap='RdBu_r', vmin=-0.8, vmax=0.8, annot=True, fmt='.1%', ax=ax, cbar=False)
    return ax

plot_correlations(df_artist[['AgePassing', 'JaarTop2000', 'DaysToStemperiode', 'LogPopularity', 'IsDutch']], figsize=(6, 6))

## Song

### Solo song
We hypothesize that solo songs get more of a boost than duets

In [ ]:
df.groupby(['NrArtists'])['LogBoost'].agg(['mean', 'std', 'sem', 'count'])

There is only one song that has more than 2 performers (namely, _Laat me/Vivre_ by Alderliefste, Liesbeth List and Ramses Shaffy; the latter two have passed away during the Top 2000), so let's group that together, since the errors are huge.

In [ ]:
display(df.groupby(['MultiplePerformers'])['LogBoost'].agg(['mean', 'std', 'sem', 'count']))
(df.groupby(['MultiplePerformers'])
 ['LogBoost'].agg(['mean', 'std', 'sem', 'count'])
 .plot(y='mean', yerr='sem',
       xlim=(-0.5, 1.5), kind='bar')
)

There are not a lot of duets, but there is a hint they may indeed have a smaller boost

### Popularity within artist oeuvre

In the [literature](https://link.springer.com/article/10.1007/s11002-014-9322-1) there is some work that shows more popular albums get larger boosts.

In [ ]:
ax = plot_with_trend(df, 'PopularityWithinArtist', logy=True)

We do not see that effect very strongly, but again, we are dominated by a few outliers

In [ ]:
ax = plot_with_trend(df, 'LogSongPopularityWithinArtist', logy=True)

In fact, there appears to be a contrary effect. This may be explained, however, by a) our assumptions of the voting distribution and/or b) collinearity with other effects

### Recency within artist

I hypothesize the more recent work may receive more of a boost, since the older work may be somewhat forgotten by now

In [ ]:
ax = plot_with_trend(df.query('NrsBeforeDeath > 2'), 'RecencyWithinArtist', logy=True)

We do not see that effect.